In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok -q

In [ ]:
!pip install uvicorn
!pip install fastapi
!pip install python-multipart
!pip install simple_lama_inpainting
!pip install diffusers
!pip install python-multipart opencv-python-headless pillow
!pip install futures

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116931 sha256=cef8f449d519e235b5863be81ce3095a3d74e6478304264d3b42c745732548b8
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for futures: filename=futures-3.0.5-py3-none-any.whl size=14069 sha256=3c83c8fe21ff8df240865b62f51123470c10f1e285aae632e3b136433e0f3fa4
  Stored in directory: /root/.cache/pip/wheels/ef/af/93/48739d464ba97d4cdc77c627d282f9794c8d276e42aaa92160
Successfully built futures


In [ ]:
import os

# DeepFillv2 セットアップ
HOME = "/content"
%cd {HOME}
!git clone https://github.com/vrindaprabhu/deepfillv2_colab.git
!gdown "https://drive.google.com/u/0/uc?id=1uMghKl883-9hDLhSiI8lRbHCzCmmRwV-&export=download"
!mv /content/deepfillv2_WGAN_G_epoch40_batchsize4.pth deepfillv2_colab/model/deepfillv2_WGAN.pth

# GroundingDINO セットアップ
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

# GroundingDINOのコンフィグとウェイトのダウンロード
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

!mkdir -p {HOME}/weights
%cd {HOME}/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))
%cd {HOME}/GroundingDINO

/content
Cloning into 'deepfillv2_colab'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 99 (delta 2), reused 1 (delta 1), pack-reused 96 (from 1)
Receiving objects: 100% (99/99), 571.56 KiB | 19.71 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Downloading...
From: https://drive.google.com/u/0/uc?id=1uMghKl883-9hDLhSiI8lRbHCzCmmRwV-&export=download
To: /content/deepfillv2_WGAN_G_epoch40_batchsize4.pth
100% 64.8M/64.8M [00:00<00:00, 68.1MB/s]
/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 463 (delta 175), reused 136 (delta 136), pack-reused 223 (from 1)
Receiving objects: 100% (463/463), 12.87 MiB | 9.48 MiB/s, done.
Resolving deltas: 100% (241/241), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━

In [ ]:
import os
import supervision as sv
from PIL import Image
import numpy as np
import cv2
import pycocotools.mask as mask_util
from groundingdino.util.inference import load_model, load_image, predict, annotate
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from fastapi.responses import HTMLResponse
import shutil
import cv2
import json
import numpy as np
import os
from pathlib import Path
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image
from simple_lama_inpainting import SimpleLama
import numpy as np
from groundingdino.util.inference import load_model, load_image, predict
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MiniBatchKMeans, Birch, SpectralClustering, MeanShift, OPTICS
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier
import torch
from torchvision import transforms
from PIL import Image
import threading
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
import shutil
import cv2
import json
import numpy as np
import os
from pathlib import Path
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image
from simple_lama_inpainting import SimpleLama
import numpy as np
from groundingdino.util.inference import load_model, load_image, predict
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MiniBatchKMeans, Birch, SpectralClustering, MeanShift, OPTICS
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier
import torch
from torchvision import transforms
from PIL import Image
import concurrent.futures  # 追加
from pathlib import Path
import os
import concurrent.futures
from typing import Tuple
from types import SimpleNamespace
import subprocess
import shutil
import os
HOME = "/content"
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
detection_model = load_model(CONFIG_PATH, WEIGHTS_PATH)

final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


In [ ]:
# 認証トークンをファイルから読み込み
with open('/content/drive/MyDrive/ngrok_token.txt', 'r') as file:
    ngrok_auth_token = file.read().strip()

# 認証トークンを使用して ngrok に設定
!ngrok authtoken {ngrok_auth_token}


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from PIL import Image, ImageFilter
from concurrent.futures import ThreadPoolExecutor


ここからが本番コード
上は、セットアップコード


In [ ]:
import math
dangerarray=[10,30,90,50,80,20,40,70,100,60]#ここに各クラスターの危険度を設定しておく
#ここで認識する精度を上げたり下げたりできる

thresholds = {
    'text': 0.1,
    'Name tag': 0.1,
    'License plate': 0.3,
    'Mail': 0.3,
    'Documents': 0.3,
    'QR codes': 0.4,
    'barcodes': 0.4,
    'Map': 0.5,
    'Digital screens': 0.6,
    'information board': 0.5,
    'signboard': 0.3,
    'poster': 0.8,
    'sign': 0.3,
    'logo': 0.3,
    'card': 0.4,
    'window': 0.2,
    'mirror': 0.2,
    'Famous landmark': 0.7,
    'cardboard': 0.6,
    'manhole': 0.6,
    'utility pole': 0.7
}

# Define paths
HOME = "/content"
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
from PIL import Image

def is_bright(pixel):
    # ピクセルの輝度を計算して明るさを判定する
    r, g, b = pixel
    brightness = (0.299 * r + 0.587 * g + 0.114 * b)  # 輝度の計算
    return brightness > 127  # 閾値を127に設定

def analyze_mask_brightness(original_image_path, mask_image_path):
    # 画像を開く
    original_img = Image.open(original_image_path).convert('RGB')
    mask_img = Image.open(mask_image_path).convert('L')  # グレースケールに変換

    width, height = original_img.size

    if mask_img.size != (width, height):
        print("エラー: マスク画像と元画像のサイズが一致していません。")
        return

    # 明るいピクセルと暗いピクセルのカウント
    bright_count = 0
    dark_count = 0

    for y in range(height):
        for x in range(width):
            mask_value = mask_img.getpixel((x, y))
            if mask_value > 127:  # マスクが白（対象領域）ならば
                pixel = original_img.getpixel((x, y))
                if is_bright(pixel):
                    bright_count += 1
                else:
                    dark_count += 1

    # 明るさの結果を判定
    brightness_result = 1 if bright_count > dark_count else 2

    return brightness_result

def classify_mask_size(mask_image_path, small_threshold, medium_threshold, large_threshold):
    # マスク画像を開く
    mask_img = Image.open(mask_image_path).convert('L')  # グレースケールに変換

    width, height = mask_img.size
    total_pixels = width * height
    white_pixel_count = 0

    # マスク画像の白いピクセルをカウント
    for y in range(height):
        for x in range(width):
            mask_value = mask_img.getpixel((x, y))
            if mask_value > 127:  # 白いピクセルと判断
                white_pixel_count += 1

    # 白いピクセルの割合を計算
    mask_area_ratio = (white_pixel_count / total_pixels) * 100

    # マスクサイズを分類
    if mask_area_ratio <= small_threshold:
        size_category = 1  # すごく小さい
    elif mask_area_ratio <= medium_threshold:
        size_category = 2  # 小さい
    elif mask_area_ratio <= large_threshold:
        size_category = 3  # 大きい
    else:
        size_category = 4  # すごく大きい

    return size_category

def analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold):
    # マスクの大きさを判定
    size_category = classify_mask_size(mask_image_path, small_threshold, medium_threshold, large_threshold)

    # マスク部分の明るさを判定
    brightness_result = analyze_mask_brightness(original_image_path, mask_image_path)

    # 結果を出力
    size_text = {1: "すごく小さい", 2: "小さい", 3: "大きい", 4: "すごく大きい"}
    print(f"マスクの大きさ: {size_text[size_category]} ({size_category})")
    print(f"マスクの明るさ: {brightness_result}")
    result={
            'size':size_category,
            'brightness':brightness_result
            }
    return result


def quantize_model(model):
    # モデルを動的量子化
    model_quantized = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    return model_quantized
#この下で消去対象を決定
def decide_to_object(risk_level):
    tex = [
        'text','Name tag', 'License plate', 'Mail', 'Documents', 'QR codes',
        'barcodes', 'Map', 'Digital screens', 'information board',
        'signboard', 'poster', 'sign', 'logo', 'card', 'window', 'mirror',
        'Famous landmark', 'cardboard', 'manhole', 'utility pole'

    ]
    #この配列の要素の順番を変えると消える順番が変わる。
    risk_level = int(risk_level / 20)*(len(tex)/10)#個数決定(1/2)
    return tex[:int(risk_level)+1]

def create_mask(image, x1, y1, x2, y2):
    # Create a black image with the same size as the input image
    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    # Draw a white rectangle on the mask where the object is located
    cv2.rectangle(mask, (int(x1), int(y1)), (int(x2), int(y2)), 255, -1)

    return mask

import numpy as np
import matplotlib.pyplot as plt

#この下のコードは特定の領域をマスクしないタイプのコード
def special_process_image(risk_level, image_path, point1, point2, thresholds=None):
    if thresholds is None:
        thresholds = {}

    # Generate the threshold adjustments based on risk level
    def logistic_decay(risk_level, k=0.1, r0=50):
        return 1 / (1 + np.exp(-k * (risk_level - r0)))

    threshold_results = {key: [] for key in thresholds.keys()}
    decay_factor = logistic_decay(risk_level)

    # Adjust thresholds for the given risk level
    adjusted_thresholds = {key: max(value - decay_factor + 0.8, 0.01) / 2 for key, value in thresholds.items()}
    detection_model = load_model(CONFIG_PATH, WEIGHTS_PATH)
    IMAGE_PATH = image_path
    LABELS = decide_to_object(risk_level)
    image_source, image = load_image(IMAGE_PATH)

    all_boxes = []
    all_logits = []

    for label in LABELS:
        boxes, logits, _ = predict(
            model=detection_model,
            image=image,
            caption=label,
            box_threshold=0.3,
            text_threshold=0.3
        )
        all_boxes.extend(boxes)
        all_logits.extend(logits)

    # Convert PyTorch tensor to NumPy array and ensure it's uint8
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = np.clip(image_np * 255, 0, 255).astype(np.uint8)  # Ensure values are in [0, 255]

    # Create a copy of the image for debugging (to draw the rectangle on)
    debug_image = image_np.copy()

    # Initialize mask as black
    mask = np.zeros((image_np.shape[0], image_np.shape[1]), dtype=np.uint8)

    for i, box in enumerate(all_boxes):
        x, y, w, h = box
        confidence = all_logits[i]  # Get the confidence score
        object_type = LABELS[i % len(LABELS)]  # Determine the object type

        # Use the threshold specific to the object type, default to 0.5 if not provided
        threshold = adjusted_thresholds.get(object_type, 0.5)
        print(threshold)
        if confidence >= threshold:
            x1 = int((x - w / 2) * image_np.shape[1])
            y1 = int((y - h / 2) * image_np.shape[0])
            x2 = int((x + w / 2) * image_np.shape[1])
            y2 = int((y + h / 2) * image_np.shape[0])

            # Ensure coordinates are within the image dimensions
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(image_np.shape[1], x2), min(image_np.shape[0], y2)

            mask = create_mask(image_np, x1, y1, x2, y2) | mask

    # Convert the two points from relative (0-1) values to absolute pixel coordinates
    p1_x, p1_y = int(point1[0] * image_np.shape[1]), int(point1[1] * image_np.shape[0])
    p2_x, p2_y = int(point2[0] * image_np.shape[1]), int(point2[1] * image_np.shape[0])

    # Ensure coordinates are within the image dimensions
    x_min = max(0, min(p1_x, p2_x))
    x_max = min(image_np.shape[1], max(p1_x, p2_x))
    y_min = max(0, min(p1_y, p2_y))
    y_max = min(image_np.shape[0], max(p1_y, p2_y))

    # Set the area defined by the rectangle to 0 (black)
    mask[y_min:y_max, x_min:x_max] = 0

    # Draw a white rectangle for debugging on the original image
    cv2.rectangle(debug_image, (x_min, y_min), (x_max, y_max), (255, 255, 255), 2)  # 2 is the line thickness

    # Save the debug image with the white rectangle
    debug_image_pil = Image.fromarray(debug_image)
    debug_image_pil.save("/content/debug_image_with_rectangle.jpg")

    # Save the mask as an image
    final_image_pil = Image.fromarray(mask)
    final_image_pil.save("/content/final_mask.jpg")

    return "/content/final_mask.jpg"

#YOLOの本番バージョンコード、マスク画像作成。速度がめっちゃ早い
def special_process_image_yolo(risk_level, image_path, point1, point2, thresholds=None):
    if thresholds is None:
        thresholds = {
            'text': 0.01,
            'Name tag': 0.1,
            'License plate': 0.3,
            'Mail': 0.3,
            'Documents': 0.3,
            'QR codes': 0.4,
            'barcodes': 0.4,
            'Map': 0.5,
            'Digital screens': 0.6,
            'information board': 0.5,
            'signboard': 0.3,
            'poster': 0.8,
            'sign': 0.3,
            'logo': 0.3,
            'card': 0.4,
            'window': 0.2,
            'mirror': 0.2,
            'Famous landmark': 0.7,
            'cardboard': 0.6,
            'manhole': 0.6,
            'utility pole': 0.7
        }

    # タイムスタンプを作成
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

    # Generate the threshold adjustments based on risk level
    def logistic_decay(risk_level, k=0.1, r0=50):
        return 1 / (1 + np.exp(-k * (risk_level - r0)))

    threshold_results = {key: [] for key in thresholds.keys()}
    decay_factor = logistic_decay(risk_level)

    # Adjust thresholds for the given risk level
    adjusted_thresholds = {key: max(value - decay_factor + 0.8, 0.01) / 2 for key, value in thresholds.items()}

    # YOLOv8モデルを読み込む
    model = YOLO('/content/drive/MyDrive/学習データ/yolov8n.pt')  # モデルのパスを適宜変更してください

    # 画像を読み込む
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 推論を実行する
    results = model(image_rgb)

    # 検出結果からボックスと信頼スコアを取得
    detections = results[0].boxes  # ボックス情報を含む結果のリスト

    # Convert image to NumPy array and ensure it's uint8
    image_np = np.array(image_rgb, dtype=np.uint8)

    # Create a copy of the image for debugging (to draw the rectangle on)
    debug_image = image_np.copy()

    # Initialize mask as black
    mask = np.zeros((image_np.shape[0], image_np.shape[1]), dtype=np.uint8)

    for box in detections:
        x, y, w, h = box.xywh[0]  # ボックスの中心座標と幅・高さ
        confidence = box.conf[0]  # ボックスの信頼度
        class_id = box.cls[0]  # オブジェクトのクラスID
        object_type = model.names[int(class_id)]  # オブジェクトのタイプを取得

        # クラス名に基づいたしきい値を使用する
        threshold = adjusted_thresholds.get(object_type, 0.5)  # デフォルト値は0.5

        if confidence >= threshold:
            x1 = int((x - w / 2) * image_np.shape[1])
            y1 = int((y - h / 2) * image_np.shape[0])
            x2 = int((x + w / 2) * image_np.shape[1])
            y2 = int((y + h / 2) * image_np.shape[0])

            # Ensure coordinates are within the image dimensions
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(image_np.shape[1], x2), min(image_np.shape[0], y2)

            mask = create_mask(image_np, x1, y1, x2, y2) | mask

    # Convert the two points from relative (0-1) values to absolute pixel coordinates
    p1_x, p1_y = int(point1[0] * image_np.shape[1]), int(point1[1] * image_np.shape[0])
    p2_x, p2_y = int(point2[0] * image_np.shape[1]), int(point2[1] * image_np.shape[0])

    # Ensure coordinates are within the image dimensions
    x_min = max(0, min(p1_x, p2_x))
    x_max = min(image_np.shape[1], max(p1_x, p2_x))
    y_min = max(0, min(p1_y, p2_y))
    y_max = min(image_np.shape[0], max(p1_y, p2_y))

    # Set the area defined by the rectangle to 0 (black)
    mask[y_min:y_max, x_min:x_max] = 0

    # Draw a white rectangle for debugging on the original image
    cv2.rectangle(debug_image, (x_min, y_min), (x_max, y_max), (255, 255, 255), 2)  # 2 is the line thickness

    # Save the debug image with the white rectangle, including the timestamp in the filename
    debug_image_pil = Image.fromarray(debug_image)
    debug_image_pil.save(f"/content/debug_image_with_rectangle_{timestamp}.jpg")

    # Save the mask as an image, including the timestamp in the filename
    final_image_pil = Image.fromarray(mask)
    final_image_pil.save(f"/content/final_mask_{timestamp}.jpg")

    return f"/content/final_mask_{timestamp}.jpg"


#この下のコードは普通タイプのコード

def process_image(risk_level, image_path, thresholds=None):
    if thresholds is None:
        thresholds = {}

    # Generate the threshold adjustments based on risk level
    def logistic_decay(risk_level, k=0.1, r0=50):
        return 1 / (1 + np.exp(-k * (risk_level - r0)))

    threshold_results = {key: [] for key in thresholds.keys()}
    decay_factor = logistic_decay(risk_level)

    # Adjust thresholds for the given risk level
    adjusted_thresholds = {key: max(value - decay_factor + 0.8, 0.01) / 2 for key, value in thresholds.items()}
    print(adjusted_thresholds)
    detection_model = load_model(CONFIG_PATH, WEIGHTS_PATH)
    risk_level = int(risk_level)
    IMAGE_PATH = image_path
    LABELS = decide_to_object(risk_level)
    image_source, image = load_image(IMAGE_PATH)

    all_boxes = []
    all_logits = []

    for label in LABELS:
        boxes, logits, _ = predict(
            model=detection_model,
            image=image,
            caption=label,
            box_threshold=0.3,
            text_threshold=0.3
        )
        all_boxes.extend(boxes)
        all_logits.extend(logits)

    # Convert PyTorch tensor to NumPy array and ensure it's uint8
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = np.clip(image_np * 255, 0, 255).astype(np.uint8)  # Ensure values are in [0, 255]

    # Initialize mask as black
    mask = np.zeros((image_np.shape[0], image_np.shape[1]), dtype=np.uint8)

    for i, box in enumerate(all_boxes):
        x, y, w, h = box
        confidence = all_logits[i]  # Get the confidence score
        object_type = LABELS[i % len(LABELS)]  # Determine the object type

        # Use the threshold specific to the object type, default to the adjusted threshold if not provided
        threshold = adjusted_thresholds.get(object_type, 0.5)

        if confidence >= threshold:
            # Adjust the box coordinates, convert to integers
            x1 = int((x - w / 2) * image_np.shape[1])
            y1 = int((y - h / 2) * image_np.shape[0])
            x2 = int((x + w / 2) * image_np.shape[1])
            y2 = int((y + h / 2) * image_np.shape[0])

            # Ensure coordinates are within the image dimensions
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(image_np.shape[1], x2), min(image_np.shape[0], y2)

            mask = create_mask(image_np, x1, y1, x2, y2) | mask

    # Save the mask as an image
    final_image_pil = Image.fromarray(mask)
    final_image_pil.save("/content/final_mask.jpg")

    return "/content/final_mask.jpg"


#この下は、openCV
def inpaint_image_with_mask(image_path, mask_path, output_path, inpaint_radius=5, inpaint_method=cv2.INPAINT_TELEA):
    """
    マスク画像を使用して元画像のインペイントを行う関数。

    Parameters:
    - image_path: 元画像のパス
    - mask_path: マスク画像のパス（修復したい領域が白、その他が黒）
    - output_path: インペイント結果の出力パス
    - inpaint_radius: インペイントの半径（デフォルトは5）
    - inpaint_method: インペイントのアルゴリズム（デフォルトはcv2.INPAINT_TELEA）

    Returns:
    - inpainted_image: インペイントされた画像
    """
    # 画像とマスクを読み込み
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # マスクはグレースケールで読み込み

    # マスク画像が正常に読み込めたかチェック
    if image is None:
        raise ValueError(f"元画像が見つかりません: {image_path}")
    if mask is None:
        raise ValueError(f"マスク画像が見つかりません: {mask_path}")

    # マスク画像が元画像と同じサイズでない場合、リサイズ
    if image.shape[:2] != mask.shape[:2]:
        print(f"マスク画像のサイズを元画像に合わせてリサイズします: {mask.shape} -> {image.shape[:2]}")
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

    # インペイント処理
    inpainted_image = cv2.inpaint(image, mask, inpaint_radius, inpaint_method)

    # インペイント結果を保存
    cv2.imwrite(output_path, inpainted_image)

    return inpainted_image

#この下はlama-simpleのinpaint
def inpaint_image_with_mask1(img_path, mask_path, output_path, resize_factor=0.5):
    print('lama')
    # 画像とマスクを読み込み
    image = Image.open(img_path)
    mask = Image.open(mask_path).convert('L')  # マスクをグレースケールに変換

    # 画像とマスクのサイズを合わせる
    mask = mask.resize(image.size, Image.NEAREST)
   #マスクのエッジをぼかす (Gaussian Blur)
    blurred_mask = mask.filter(ImageFilter.GaussianBlur(radius=3))  #半径3ピクセルでぼかし
    # SimpleLama インスタンスを作成
    simple_lama = SimpleLama()

    # インペイントの実行
    result = simple_lama(image, mask)

    # 出力画像をリサイズ
    new_size = (int(result.width * resize_factor), int(result.height * resize_factor))
    result = result.resize(new_size, Image.ANTIALIAS)

    # 結果を保存
    result.save(output_path)
    print(f"Inpainted image saved at {output_path}")
    return output_path



#この下はStable Diffusion
def load_inpainting_pipeline():
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16
    )
    pipe = pipe.to("cuda")
    return pipe

def inpaint_images(image_paths, mask_paths, output_dir="output_images"):
    #パイプラインの読み込み
    pipe = load_inpainting_pipeline()

    #出力ディレクトリの作成
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    #並列処理でインペインティングを実行
    with ThreadPoolExecutor() as executor:
        futures = []
        for i, (image_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
            #画像とマスクを読み込み
            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            #画像とマスクの読み込みチェック
            if image is None:
                raise ValueError(f"{image_path} が正しく読み込まれていません。ファイルパスを確認してください。")
            if mask is None:
                raise ValueError(f"{mask_path} が正しく読み込まれていません。ファイルパスを確認してください。")

            #マスクと画像のサイズ確認
            if mask.shape[:2] != image.shape[:2]:
                raise ValueError(f"マスク {mask_path} のサイズが画像 {image_path} のサイズと一致しません。")

            #画像とマスクをPIL形式に変換
            image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            mask_pil = Image.fromarray(mask)

            #インペインティング処理の実行を非同期で開始
            futures.append(executor.submit(
                lambda img, m, idx: pipe(
                    prompt="Fill the masked area with contextually appropriate background or scenery that matches the surrounding area.",
                    negative_prompt="Exclude people, human figures, arms, and any human body parts from the generated content.",
                    image=img,
                    mask_image=m,
                    num_inference_steps=100,  #ステップ数を増加
                    guidance_scale=7.5  #高いガイダンススケールを設定
                ).images[0], image_pil, mask_pil, i
            ))

        #結果を取得して保存
        for i, future in enumerate(futures):
            result = future.result()

            #結果のリサイズと保存
            result_image = result.resize((image.shape[1], image.shape[0]), Image.LANCZOS)
            output_image_path = os.path.join(output_dir, f"output_{i+1}.png")
            result_image.save(output_image_path)

            #処理が完了したことを通知
            print(f"{image_paths[i]} と {mask_paths[i]} のペアの処理が完了しました。結果は {output_image_path} に保存されました。")
#下記はDeepFillv2のコード



app = FastAPI()
# CORSミドルウェアの追加
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # ここを適切なオリジンに設定することもできます
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 保存先のディレクトリを指定
SAVE_DIR = Path("/content/saved_images")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

def save_image(file, filename):
    """画像ファイルを指定ディレクトリに保存"""
    filepath = SAVE_DIR / filename
    with open(filepath, "wb") as buffer:
        shutil.copyfileobj(file, buffer)
    return filepath

@app.post("/create-mask-and-inpaint-opencv")
async def create_mask_and_inpaint_opencv(image: UploadFile = File(...), risk_level: int = Form(...)):
    input_path = save_image(image.file, "input.jpg")
    mask_path = process_image(risk_level, input_path, thresholds=thresholds)

    output_path = SAVE_DIR / "output_opencv.jpg"
    # OpenCVでインペイント
    inpaint_image_with_mask(input_path, mask_path, output_path)

    return FileResponse(output_path)

@app.post("/create-mask-and-inpaint-simple-lama")
async def create_mask_and_inpaint_simple_lama(image: UploadFile = File(...), risk_level: int = Form(...)):
    input_path = save_image(image.file, "input.jpg")
    mask_path =  process_image(risk_level, input_path, thresholds=thresholds)# マスク画像を作成
    output_path = SAVE_DIR / "output_simple_lama.jpg"
    # SimpleLamaでインペイント
    inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)

    return FileResponse(output_path)


#下のendpointは特定領域をマスクしないタイプのもの


#下記はDeepFillv2



# マスクを画像サイズにリサイズする関数
def resize_mask_to_image(mask_image_path, target_image_path):
    """
    マスク画像のサイズを元画像のサイズに合わせる
    """
    # 元画像とマスクを読み込む
    target_image = Image.open(target_image_path)
    mask_image = Image.open(mask_image_path)

    # マスクを元画像のサイズにリサイズ
    resized_mask = mask_image.resize(target_image.size, Image.NEAREST)

    return resized_mask


class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def copy_image_to_folder(image_path, save_as_input):
    """
    Copies the given image to the appropriate folder based on the save_as_input flag.

    Parameters:
    image_path (str): The path of the image to be copied.
    save_as_input (int): 0 to save as input image, 1 to save as mask image.
    """
    if not os.path.exists(image_path):
        print(f"Error: The file {image_path} does not exist!")
        raise StopExecution

    # Set the destination folder and file name based on the save_as_input flag
    if save_as_input == 0:
        destination_path = "/content/deepfillv2_colab/input/input_img.png"
        print(f"Saving image as input: {destination_path}")
    elif save_as_input == 1:
        destination_path = "/content/deepfillv2_colab/input/mask.png"
        print(f"Saving image as mask: {destination_path}")
    else:
        print("Error: Invalid save_as_input value. It must be 0 (input) or 1 (mask).")
        raise StopExecution

    # Copy the image to the destination
    shutil.copy(image_path, destination_path)
    print(f"Image {image_path} copied to {destination_path} successfully!")


def run_python_script(script_path):
    try:
        # Pythonスクリプトを新しいプロセスとして実行
        result = subprocess.run(['python', script_path], check=True, text=True, capture_output=True)
        # 実行が成功した場合、標準出力を表示
        print("Script output:", result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running script: {e}")
        print("Script stderr:", e.stderr)



@app.post("/create-mask-and-inpaint-deepfillv2")
# エンドポイントの実装
async def deepfillv2_inpaint_endpoint(
    image: UploadFile = File(...),
    risk_level: int = Form(...)
):
    input_path = save_image(image.file, "input.jpg")
    save_as_input=0
    copy_image_to_folder(input_path, save_as_input)
    input_path=process_image(risk_level,input_path, thresholds=thresholds)
    save_as_input=1#option
    copy_image_to_folder(input_path, save_as_input)
    os.chdir(f"{HOME}/deepfillv2_colab")#一度ディレクトリを変更
    run_python_script('/content/deepfillv2_colab/inpaint.py')#プロセス用のコードを呼び出す
    #!python inpaint.py
    os.chdir(f"{HOME}/GroundingDINO") #元に戻す
    output_image_path='/content/deepfillv2_colab/output/inpainted_img.png'
    return FileResponse(output_image_path)


@app.post("/create-mask-and-inpaint-stable-diffusion")
async def create_mask_and_inpaint_stable_diffusion(image: UploadFile = File(...), risk_level: int = Form(...)):
    input_path = save_image(image.file, "input.jpg")
    mask_path = process_image(risk_level, input_path, thresholds=thresholds) # マスク画像を作成

    output_path = SAVE_DIR / "output_stable_diffusion.png"

    # 保存された画像の有効性を確認
    validate_image(input_path)


    # マスクと元画像のサイズが一致しない場合はリサイズ
    resize_mask_if_needed(mask_path, input_path)

    # Stable Diffusionでインペイント
    inpaint_images([input_path], [mask_path], output_dir=SAVE_DIR)

    return FileResponse(SAVE_DIR / "output_1.png")

def validate_image(filepath):
    try:
        with Image.open(filepath) as img:
            img.verify()
        print(f"Image at {filepath} is valid.")
    except Exception as e:
        print(f"Invalid image file at {filepath}: {e}")
        raise ValueError(f"Invalid image file: {e}")

def resize_mask_if_needed(mask_path, input_path):
    with Image.open(input_path) as input_image:
        with Image.open(mask_path) as mask_image:
            if input_image.size != mask_image.size:
                print(f"Resizing mask from {mask_image.size} to {input_image.size}")
                resized_mask = mask_image.resize(input_image.size)
                resized_mask.save(mask_path)

# ベクトル化対象のオブジェクトリスト
TEXT_PROMPTS = [
     'text','Name tag', 'License plate', 'Mail', 'Documents', 'QR codes',
        'barcodes', 'Map', 'Digital screens', 'information board',
        'signboard', 'poster', 'sign', 'logo', 'card', 'window', 'mirror',
        'Famous landmark', 'cardboard', 'manhole', 'utility pole'
]
BOX_THRESHOLD = 0.3
TEXT_THRESHOLD = 0.3

# クラスタリング結果をJSONファイルから読み込む関数
def load_sums_from_json(filepath):
    with open(filepath, 'r') as json_file:
        sums = json.load(json_file)
    return sums

# ベクトルデータをJSONファイルから読み込む関数
def load_vectors_from_json(filepath):
    with open(filepath, 'r') as json_file:
        data = json.load(json_file)
    return data

# 新しい画像を分類する関数
def classify_new_image(new_image_vector, sums_data, loaded_vectors, loaded_object_names, k=1):
    cluster_centers = []
    for cluster in sums_data:
        indices = [loaded_object_names.index(obj_name) for obj_name in cluster]
        cluster_vectors = np.array([loaded_vectors[obj_name] for obj_name in cluster])
        cluster_center = np.mean(cluster_vectors, axis=0)
        cluster_centers.append(cluster_center)

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(cluster_centers, range(len(cluster_centers)))

    new_image_label = knn.predict([new_image_vector])
    return new_image_label[0]

# 画像ベクトル化の処理（例としての関数）
def process_image_vec(image_path):
    detection_model = load_model(CONFIG_PATH, WEIGHTS_PATH)

    image_source, image = load_image(image_path)
    object_vector = np.zeros(len(TEXT_PROMPTS))

    for i, text_prompt in enumerate(TEXT_PROMPTS):
        boxes, logits, phrases = predict(
            model=detection_model,
            image=image,
            caption=text_prompt,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD
        )

        if len(logits) > 0:
            logits_np = np.array(logits)
            object_vector[i] = np.sum(logits_np)
    print(object_vector)
    return object_vector.tolist()

# APIのエンドポイント
@app.post("/classify-image/")
async def classify_image(file: UploadFile = File(...)):
    image_path = "/tmp/temp_image.jpg"

    # アップロードされた画像を保存
    with open(image_path, "wb") as buffer:
        buffer.write(await file.read())

    # 画像をベクトル化
    new_image_vector = process_image_vec(image_path)

    # JSONファイルからデータを読み込む
    json_filepath = "/content/drive/MyDrive/lastsum/output_vectors.json"
    loaded_data = load_vectors_from_json(json_filepath)
    loaded_vectors = {obj_name: np.array(vector) for obj_name, vector in loaded_data.items()}
    loaded_object_names = list(loaded_vectors.keys())

    # 既存のクラスタリング結果を読み込む
    sums_data = load_sums_from_json("/content/drive/MyDrive/lastsum/sums_data.json")

    # 新しい画像がどのクラスタに分類されるかを判定
    new_image_cluster = classify_new_image(new_image_vector, sums_data, loaded_vectors, loaded_object_names)

    return {"danger":dangerarray[int(new_image_cluster + 1)]}#バグったらここを＋にして


@app.post("/create-mask-and-inpaint-simple-lama-special")
async def create_mask_and_inpaint_simple_lama(
    image: UploadFile = File(...),
    risk_level: int = Form(...),
    x1: float = Form(...),
    y1: float = Form(...),
    x2: float = Form(...),
    y2: float = Form(...),
):
    # Extract points from the form data
    point1 = [x1, y1]
    point2 = [x2, y2]

    # Save the input image
    input_path = save_image(image.file, "input.jpg")
    print('1111',point1,point2)
    # Create a mask image (using the new process_image function)
    mask_path = special_process_image(risk_level, input_path, point1, point2,thresholds=thresholds)

    # Define the output path for the inpainted image
    output_path = "/content/output_simple_lama.jpg"

    # Perform inpainting with SimpleLama
    inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)

    # Return the resulting image as a response
    return FileResponse(output_path, media_type="image/jpeg", filename="output_simple_lama.jpg")




from PIL import Image

def resize_mask_to_match(image_path, mask_path):
    # オリジナル画像とマスク画像を読み込む
    original_image = Image.open(image_path)
    mask_image = Image.open(mask_path)

    # マスク画像をオリジナル画像のサイズにリサイズ
    resized_mask = mask_image.resize(original_image.size)

    # マスク画像を上書き保存
    resized_mask.save(mask_path)

@app.post("/create-mask-and-inpaint-sum")
async def create_mask_sum(image: UploadFile = File(...), risk_level: int = Form(...),
    x1: float = Form(...),
    y1: float = Form(...),
    x2: float = Form(...),
    y2: float = Form(...),):

    input_path = save_image(image.file, "/content/saved_images/input.jpg")
    default_x = 0.001
    default_y = 0.001


    point1 = [default_x if math.isnan(x1) else x1, default_y if math.isnan(y1) else y1]

    point2 = [default_x if math.isnan(x2) else x2, default_y if math.isnan(y2) else y2]


    print('1111',point1,point2)
    # Create a mask image (using the new process_image function)
    mask_path = special_process_image(risk_level, input_path, point1, point2,thresholds=thresholds)

    # Define the output path for the inpainted image
    output_path = "/content/output_simple_lama.jpg"

    # Perform inpainting with SimpleLama
    inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)

    # Return the resulting image as a response
    return FileResponse(output_path, media_type="image/jpeg", filename="output_simple_lama.jpg")
    '''
    mask_path = "/content/final_mask.jpg"  # マスク画像の保存パス
    output_path = SAVE_DIR / "output_simple_lama.jpg"

    # special_process_imageの実行
    mask_path = special_process_image(risk_level, input_path, point1, point2, thresholds=thresholds)

    # special_process_imageの後にマスク画像とオリジナル画像のサイズを揃える
    resize_mask_to_match(input_path, mask_path)

    original_image_path = input_path
    mask_image_path = mask_path
    small_threshold = 10.0   # 「すごく小さい」の上限（5%）
    medium_threshold = 25.0  # 「小さい」の上限（20%）
    large_threshold = 50.0   # 「大きい」の上限（50%）
    # マスク画像を作成

    if analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['size'] == 1:#サイズ1
      if analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==1:
        #めっちゃ小さくて明るいもの
        output_path = SAVE_DIR / "output_opencv.jpg"
        # OpenCVでインペイント
        inpaint_image_with_mask(input_path, mask_path, output_path)
      elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==2:
         #めっちゃ小さくて暗いもの
        inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)#SLI

    elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['size'] == 2: #サイズ2
      if analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==1:
        #小さくて明るいもの
        input_path = save_image(image.file, "input.jpg")
        save_as_input=0
        copy_image_to_folder(input_path, save_as_input)
        input_path=process_image(risk_level,input_path, thresholds=thresholds)
        save_as_input=1#option
        copy_image_to_folder(input_path, save_as_input)
        os.chdir(f"{HOME}/deepfillv2_colab")#一度ディレクトリを変更
        run_python_script('/content/deepfillv2_colab/inpaint.py')#プロセス用のコードを呼び出す
        #!python inpaint.py
        os.chdir(f"{HOME}/GroundingDINO") #元に戻す
        output_image_path='/content/deepfillv2_colab/output/inpainted_img.png' #DeepFillv2


      elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==2:
         #小さくて暗いもの
        inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)#SLI


    elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['size'] == 3: #サイズ3
      if analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==1:
        #おおきくて明るいもの
        input_path = save_image(image.file, "input.jpg")
        save_as_input=0
        copy_image_to_folder(input_path, save_as_input)
        input_path=process_image(risk_level,input_path, thresholds=thresholds)
        save_as_input=1#option
        copy_image_to_folder(input_path, save_as_input)
        os.chdir(f"{HOME}/deepfillv2_colab")#一度ディレクトリを変更
        run_python_script('/content/deepfillv2_colab/inpaint.py')#プロセス用のコードを呼び出す
        #!python inpaint.py
        os.chdir(f"{HOME}/GroundingDINO") #元に戻す
        output_image_path='/content/deepfillv2_colab/output/inpainted_img.png' #DeepFillv2


      elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==2:
         #おおきくて暗いもの
        inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)#SLI


    elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['size'] == 4: #サイズ4
      if analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==1:
        #すごくおおきくて明るいもの
        inpaint_images([input_path], [mask_path], output_dir=SAVE_DIR)#diffusion

      elif analyze_mask_combined(original_image_path, mask_image_path, small_threshold, medium_threshold, large_threshold)['brightness']==2:
         #すごくおおきくて暗いもの
        inpaint_image_with_mask1(input_path, mask_path, output_path, resize_factor=1)#SLI


    return FileResponse(output_path)
'''

@app.get("/", response_class=HTMLResponse)
async def read_root():
    html_content = """
 <!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>画像処理アプリ</title>
    <!-- Bootstrap CSS -->
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
    <!-- jQuery UI CSS -->
    <link rel="stylesheet" href="https://code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css" rel="stylesheet">
    <style>
        body {
            background-color: #f0f0f5;
            color: #333;
            text-align: center;
            padding: 40px 20px;
        }
        h1 {
            color: #555;
            margin-bottom: 30px;
            font-weight: bold;
        }
        .image-preview, .processed-preview {
            max-width: 100%;
            height: auto;
            border-radius: 10px;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
            margin-top: 20px;
        }
        #result {
            margin-top: 40px;
            display: none;
        }
        .slider-container {
            text-align: left;
            margin-top: 20px;
        }
        .slider-label {
            font-size: 1.2rem;
            color: #333;
        }
        #slider {
            margin-top: 10px;
        }
        .btn-primary {
            background-color: #007bff;
            border-color: #007bff;
            font-size: 1.2rem;
            padding: 10px 20px;
            border-radius: 50px;
        }
        .btn-primary:hover {
            background-color: #0056b3;
            border-color: #004085;
        }
        .form-control, .custom-select {
            border-radius: 20px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        .form-control-file {
            font-size: 1rem;
        }
        .form-group {
            margin-bottom: 25px;
        }
        .btn-success {
            padding: 10px 20px;
            border-radius: 50px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1><i class="fas fa-image"></i> 画像処理アプリ - モザイクとインペイント</h1>

        <div class="form-group">
            <input type="file" id="uploadImage" class="form-control-file" accept="image/*" onchange="previewImage()">
        </div>
        <img id="uploadedImage" class="image-preview" src="#" alt="アップロードされた画像" style="display: none;">

        <div class="form-group mt-4">
            <label for="processingType">処理方法を選択:</label>
            <select id="processingType" class="custom-select">
                <option value="opencv">OpenCVインペイント</option>
                <option value="simple_lama">Simple Lamaインペイント</option>
                <option value="stable_diffusion">Stable Diffusionインペイント</option>
            </select>
        </div>

        <div class="slider-container">
            <label for="riskLevel" class="slider-label">リスクレベル (0-100): <span id="riskLevelLabel">50</span></label>
            <div id="slider"></div>
        </div>

        <button class="btn btn-primary mt-4" onclick="processImage()">処理開始</button>

        <div id="result" class="mt-5">
            <h2>処理結果:</h2>
            <img id="processedImage" class="processed-preview" src="" alt="">
            <a id="downloadLink" class="btn btn-success mt-3" href="#" download="processed_image.jpg">処理された画像をダウンロード</a>
        </div>
    </div>

    <!-- jQuery and Bootstrap JS -->
    <script src="https://code.jquery.com/jquery-3.5.1.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
    <!-- jQuery UI -->
    <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

    <script>
        $(function() {
            // スライダーの設定
            $("#slider").slider({
                range: "min",
                value: 50, // 初期値
                min: 0,
                max: 100,
                slide: function(event, ui) {
                    $("#riskLevelLabel").text(ui.value);
                }
            });
        });

        function previewImage() {
            const fileInput = document.getElementById('uploadImage');
            const uploadedImage = document.getElementById('uploadedImage');

            if (fileInput.files && fileInput.files[0]) {
                const reader = new FileReader();
                reader.onload = function (e) {
                    uploadedImage.src = e.target.result;
                    uploadedImage.style.display = 'block';
                };
                reader.readAsDataURL(fileInput.files[0]);
            }
        }

        function processImage() {
            const fileInput = document.getElementById('uploadImage');
            const processingType = document.getElementById('processingType').value;
            const riskLevel = $("#slider").slider("value");  // スライダーから値を取得
            const resultDiv = document.getElementById('result');
            const processedImage = document.getElementById('processedImage');
            const downloadLink = document.getElementById('downloadLink');

            if (fileInput.files.length === 0) {
                alert("画像を選択してください。");
                return;
            }

            const file = fileInput.files[0];
            const formData = new FormData();
            formData.append('image', file);
            formData.append('risk_level', riskLevel);  // リスクレベルを追加

            let apiEndpoint;
            if (processingType === "opencv") {
                apiEndpoint = "/create-mask-and-inpaint-opencv";
            } else if (processingType === "simple_lama") {
                apiEndpoint = "/create-mask-and-inpaint-simple-lama";
            } else if (processingType === "stable_diffusion") {
                apiEndpoint = "/create-mask-and-inpaint-stable-diffusion";
            }

            const url = "https://wired-kitten-adequately.ngrok-free.app" + apiEndpoint;

            fetch(url, {
                method: 'POST',
                body: formData
            })
            .then(response => {
                if (!response.ok) {
                    throw new Error("Network response was not ok");
                }
                return response.blob();
            })
            .then(blob => {
                const objectURL = URL.createObjectURL(blob);
                processedImage.src = objectURL;
                downloadLink.href = objectURL;
                resultDiv.style.display = "block";
            })
            .catch(error => {
                console.error("画像処理に失敗しました。", error);
                alert("画像処理に失敗しました。");
            });
        }
    </script>
</body>
</html>


    """
    return HTMLResponse(content=html_content)


def run_fastapi():
    if __name__ == "__main__":
        nest_asyncio.apply()
        uvicorn.run(app, host="0.0.0.0", port=8000)

def run_ngrok():
    os.system("ngrok http --domain=wired-kitten-adequately.ngrok-free.app 8000")#固定URLを発行

# スレッドを作成
fastapi_thread = threading.Thread(target=run_fastapi)
ngrok_thread = threading.Thread(target=run_ngrok)

# 二つのスレッドを動かす
fastapi_thread.start()
ngrok_thread.start()
fastapi_thread.join()
ngrok_thread.join()




INFO:     Started server process [10005]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


final text_encoder_type: bert-base-uncased


[0.63905585 0.         0.         0.73494852 0.90994442 0.74501801
 0.72196484 0.33889359 0.71385443 0.80644011 0.         0.70761728
 0.71582139 0.7175144  0.         5.18829823 0.         0.74006587
 1.14363098 0.85681629 1.66955853]
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /classify-image/ HTTP/1.1" 200 OK
1111 [0.001, 0.001] [0.001, 0.001]
final text_encoder_type: bert-base-uncased


0.44100689501895424
0.44100689501895424
lama
Inpainted image saved at /content/output_simple_lama.jpg
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /create-mask-and-inpaint-sum HTTP/1.1" 200 OK


final text_encoder_type: bert-base-uncased
[0.63905585 0.         0.         0.73494852 0.90994442 0.74501801
 0.72196484 0.33889359 0.71385443 0.80644011 0.         0.70761728
 0.71582139 0.7175144  0.         5.18829823 0.         0.74006587
 1.14363098 0.85681629 1.66955853]
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /classify-image/ HTTP/1.1" 200 OK
final text_encoder_type: bert-base-uncased


...

0.005
0.005
0.05334642546214241
0.05334642546214241
0.05334642546214241
0.1033464254621424
0.1033464254621424
0.1533464254621424
0.20334642546214238
0.1533464254621424
0.05334642546214241
0.005
0.005
0.05334642546214241
0.05334642546214241
lama
Inpainted image saved at /content/output_simple_lama.jpg
INFO:     210.137.33.142:0 - "POST /create-mask-and-inpaint-sum HTTP/1.1" 200 OK


INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /classify-image/ HTTP/1.1" 422 Unprocessable Entity
final text_encoder_type: bert-base-uncased
[0.58552831 0.46228495 0.58182102 0.51741678 1.13575172 0.68388766
 0.71755737 0.60434121 0.75826496 1.04264212 0.47483221 0.66592383
 0.77476561 0.51710141 0.79674673 0.78932613 0.86282533 0.75460112
 0.50507987 0.7004106  0.70862651]
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /classify-image/ HTTP/1.1" 200 OK
1111 [0.001, 0.001] [0.001, 0.001]
final text_encoder_type: bert-base-uncased


0.249343830056226
0.249343830056226
0.34934383005622605
0.34934383005622605
0.34934383005622605
0.249343830056226
0.249343830056226
lama
Inpainted image saved at /content/output_simple_lama.jpg
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /create-mask-and-inpaint-sum HTTP/1.1" 200 OK


final text_encoder_type: bert-base-uncased
[0.58552831 0.46228495 0.58182102 0.51741678 1.13575172 0.68388766
 0.71755737 0.60434121 0.75826496 1.04264212 0.47483221 0.66592383
 0.77476561 0.51710141 0.79674673 0.78932613 0.86282533 0.75460112
 0.50507987 0.7004106  0.70862651]
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /classify-image/ HTTP/1.1" 200 OK
1111 [0.001, 0.001] [0.001, 0.001]
final text_encoder_type: bert-base-uncased


0.44100689501895424
lama
Inpainted image saved at /content/output_simple_lama.jpg
INFO:     2400:2200:3b5:8a8d::446d:5d18:0 - "POST /create-mask-and-inpaint-sum HTTP/1.1" 200 OK


In [ ]:
os.system("ngrok http --domain=wired-kitten-adequately.ngrok-free.app 8000")#固定URLを発行

2

In [ ]:

DIR = '/content/drive/MyDrive/学習データ'

print(sum(os.path.isfile(os.path.join(DIR, name)) for name in os.listdir(DIR)))


5633


In [ ]:
!ps aux | grep 'python'

root          67  0.0  0.0      0     0 ?        Z    Oct10   0:06 [python3] <defunct>
root          68  0.0  0.3  67728 52272 ?        S    Oct10   0:00 python3 /usr/local/bin/colab-file
root         117  0.0  1.3 575896 177940 ?       Sl   Oct10   0:08 /usr/bin/python3 /usr/local/bin/j
root        1060  0.0  0.0  18024  9556 ?        S    Oct10   0:00 python3 /opt/google/drive/drive-f
root        8542  0.0  0.1 538260 13624 ?        Sl   Oct10   0:00 /usr/bin/python3 /usr/local/lib/p
root       72952 61.7 18.2 11576148 2423456 ?    Ssl  03:51   0:11 /usr/bin/python3 -m colab_kernel_
root       72973  0.4  0.1 539284 13532 ?        Sl   03:51   0:00 /usr/bin/python3 /usr/local/lib/p
root       73118  0.0  0.0   6484  2280 ?        S    03:51   0:00 grep python


下記は、最初のクラスタリングに必要なコード。今は何も手を付けなくていい


In [ ]:
!kill 15108

/bin/bash: line 1: kill: (15108) - No such process


In [ ]:
#統合用コード,vector作成
import os
import json
import numpy as np

# ベクトル化対象のオブジェクトリスト
TEXT_PROMPTS = [
    'text', 'mirror', 'window', 'cardboard', 'poster', 'sign', 'manhole',
    'logo', 'signboard', 'utility pole','Name tag','License plate','card','Mail or envelope',
    'Digital screens','QR codes','barcodes','documents','information board','Map',
    'Famous landmark'
]
BOX_THRESHOLD = 0.3
TEXT_THRESHOLD = 0.3

def process_image_vec(image_path):
    image_source, image = load_image(image_path)
    object_vector = np.zeros(len(TEXT_PROMPTS))

    for i, text_prompt in enumerate(TEXT_PROMPTS):
        boxes, logits, phrases = predict(
            model=detection_model,
            image=image,
            caption=text_prompt,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD
        )

        if len(logits) > 0:
            logits_np = np.array(logits)
            object_vector[i] = np.sum(logits_np)
    print(object_vector)
    return object_vector.tolist()

def process_images_in_folder(folder_path, output_json_path):
    results = {}

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            object_vector = process_image_vec(image_path)
            results[filename] = object_vector

    with open(output_json_path, 'w') as f:
        json.dump(results, f, indent=4)

# 画像が格納されているフォルダのパスと出力JSONファイルのパス
folder_path = "/content/drive/MyDrive/lastsum"
output_json_path = "/content/drive/MyDrive/lastsum/output_vectors.json"

# フォルダ内の画像を処理してJSONに保存
process_images_in_folder(folder_path, output_json_path)


[1.49639297 0.53853089 0.68880463 0.99608672 1.06955934 0.96696973
 1.14053011 0.56766939 0.78465068 1.0010705  0.8178345  0.440348
 0.98328477 1.37204599 0.92802715 0.5291034  0.76120877 1.04426539
 0.98724872 1.01390123 0.87089682]
[0.70044339 0.30440947 0.68930662 0.40916607 0.55707008 0.
 0.         0.71923631 0.         0.46680602 0.31489816 0.
 0.         0.69141614 0.35253656 0.32804587 0.68376303 0.42671692
 0.64998502 0.         0.76780516]
[1.22471309 0.6068123  0.64947981 1.25715756 0.8556605  0.4840408
 0.65523833 0.96553457 0.         0.70873404 0.66456181 1.04295528
 0.75752252 1.25871253 0.86201441 0.82422018 0.8417719  0.63659412
 0.8904956  0.73832071 0.92354989]
[0.47478062 0.         1.07758141 0.34041968 0.43952766 0.
 0.57516688 0.         0.         1.18100524 0.40632349 0.39842549
 0.         0.84396023 0.31448552 0.76259035 0.74724555 0.36352268
 0.41878954 0.34494674 0.38678753]


KeyboardInterrupt: 

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MiniBatchKMeans, Birch, SpectralClustering, MeanShift, OPTICS
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

def load_from_json(filepath):
    with open(filepath, 'r') as json_file:
        data = json.load(json_file)
    return list(data.values()), list(data.keys())

# JSONファイルのパスを入力
json_filepath = "/content/drive/MyDrive/lastsum/output_vectors.json"  # JSONデータが保存されたファイルパス

# JSONファイルからデータを読み込む
loaded_vectors, loaded_object_names = load_from_json(json_filepath)
X = np.array(loaded_vectors)
num = len(loaded_vectors)

# クラスタリング手法のリスト
clustering_methods = [
    ('KMeans', KMeans),
    ('MiniBatchKMeans', MiniBatchKMeans),
    ('AgglomerativeClustering', AgglomerativeClustering),
    ('DBSCAN', DBSCAN),
    ('Birch', Birch),
    ('SpectralClustering', SpectralClustering),
    ('MeanShift', MeanShift),
    ('OPTICS', OPTICS)
]

# クラスタリング手法を選択
print("Select a clustering method:")
for i, (name, method) in enumerate(clustering_methods):
    print(f"{i}: {name}")
method_index = int(input("Enter the number of the clustering method: "))
method_name, ClusteringMethod = clustering_methods[method_index]

# クラスタ数を選択する範囲
k_range = range(8, min(100, num))
# シルエットスコアの変化量を使って最適なクラスタ数を見つける
silhouette_scores = []

for k in k_range:
    if method_name in ['DBSCAN', 'MeanShift', 'OPTICS']:
        clustering = ClusteringMethod()
        labels = clustering.fit_predict(X)
        k = len(set(labels)) - (1 if -1 in labels else 0)
    else:
        clustering = ClusteringMethod(n_clusters=k)
        labels = clustering.fit_predict(X)

    if len(set(labels)) > 1:
        silhouette_scores.append(silhouette_score(X, labels))

# シルエットスコアの変化量を計算
silhouette_deltas = np.diff(silhouette_scores)

# 変化量が最大のインデックスを取得
max_delta_index = np.argmax(silhouette_deltas)

# 最適なクラスタ数
optimal_k = k_range[max_delta_index + 1]  # np.diffの結果は1つ少ないため、インデックスを調整

print(f'Optimal number of clusters based on silhouette score change: {optimal_k}')

# シルエットスコアと変化量のプロット
plt.figure()
plt.plot(k_range[:len(silhouette_scores)], silhouette_scores, marker='o', label="Silhouette Score")
plt.plot(k_range[1:len(silhouette_scores)], silhouette_deltas, marker='x', label="Silhouette Score Change", color='red')
plt.xlabel('Number of clusters')
plt.ylabel('Score / Change')
plt.title(f'Silhouette Scores and Changes for {method_name}')
plt.legend()
plt.show()

# 最適なクラスタ数でクラスタリング
if method_name in ['DBSCAN', 'MeanShift', 'OPTICS']:
    clustering = ClusteringMethod()
else:
    clustering = ClusteringMethod(n_clusters=optimal_k)
labels = clustering.fit_predict(X)

# PCAで2次元に圧縮
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# クラスタの名前を定義
cluster_names = [f'Cluster {i+1}' for i in range(len(set(labels)) - (1 if -1 in labels else 0))]

# 2次元プロット
plt.figure()
for i in range(len(cluster_names)):
    plt.scatter(X_pca[labels == i, 0], X_pca[labels == i, 1], label=cluster_names[i])
plt.title(f'{method_name} Clustering (2D projection)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

# クラスタ結果を保存
sums = []
for i in range(len(cluster_names)):
    temp = [loaded_object_names[l] for l in range(len(labels)) if labels[l] == i]
    sums.append(temp)
print(sums)

# sumsをJSONファイルに保存
def save_sums_to_json(sums, filename='sums_data.json'):
    with open(filename, 'w') as json_file:
        json.dump(sums, json_file, indent=4)

# JSONファイルの保存
output_filepath = "/content/drive/MyDrive/lastsum/sums_data.json"  # 出力するJSONファイルのパス
save_sums_to_json(sums, output_filepath)


この下は各クラスタの様子を調べるためのコード


In [ ]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt

# JSONファイルからsumsを読み込む関数
def load_sums_from_json(filepath):
    with open(filepath, 'r') as json_file:
        sums = json.load(json_file)
    return sums

# JSONファイルのパス
json_filepath = "/content/drive/MyDrive/lastsum/sums_data.json"

# sumsを読み込む
sums = load_sums_from_json(json_filepath)
print(sums)

# クラスタ番号を入力
num1 = int(input('クラスタ番号は?')) - 1
print(f'{sums[num1]}の写真がクラスタ{num1+1}に属しています')

# 画像のディレクトリパス
images_dir = "/content/drive/MyDrive/lastsum"  # 画像が保存されているディレクトリのパスを指定

# クラスタに含まれる画像を一枚ずつ表示する関数
def display_images_one_by_one(cluster_images):
    for image_filename in cluster_images:
        # ファイル名から画像番号を抽出
        image_name = image_filename.split('.')[0]
        image_path = os.path.join(images_dir, f'{image_name}.jpg')

        # 画像を読み込み
        img = Image.open(image_path)

        # 画像を大きく表示
        plt.figure(figsize=(10, 10))  # 画像の表示サイズを指定
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Image {image_name}')
        plt.show()

# クラスタの画像を一枚ずつ表示
display_images_one_by_one(sums[num1])


In [ ]:
import os
import json
import random  # ランダムに画像を選択するために必要
from PIL import Image
import matplotlib.pyplot as plt

# JSONファイルからsumsを読み込む関数
def load_sums_from_json(filepath):
    with open(filepath, 'r') as json_file:
        sums = json.load(json_file)
    return sums

# JSONファイルのパス
json_filepath = "/content/drive/MyDrive/lastsum/sums_data.json"

# sumsを読み込む
sums = load_sums_from_json(json_filepath)
print(sums)

# クラスタ番号を入力
num1 = int(input('クラスタ番号は?')) - 1
print(f'{sums[num1]}の写真がクラスタ{num1+1}に属しています')

# 画像のディレクトリパス
images_dir = "/content/drive/MyDrive/lastsum"  # 画像が保存されているディレクトリのパスを指定

# クラスタに含まれる画像からランダムに3枚表示する関数
def display_random_images(cluster_images, num_images=10):
    # クラスタ内の画像数がnum_images未満の場合、全ての画像を表示
    if len(cluster_images) < num_images:
        selected_images = cluster_images
    else:
        selected_images = random.sample(cluster_images, num_images)  # ランダムに3枚選択

    for image_filename in selected_images:
        # ファイル名から画像番号を抽出
        image_name = image_filename.split('.')[0]
        image_path = os.path.join(images_dir, f'{image_name}.jpg')

        # 画像を読み込み
        img = Image.open(image_path)

        # 画像を大きく表示
        plt.figure(figsize=(10, 10))  # 画像の表示サイズを指定
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Image {image_name}')
        plt.show()

# クラスタの画像からランダムに3枚表示
display_random_images(sums[num1])


In [ ]:
!pip freeze -> '/content/requirements.txt'